In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_18_01'
posner_id ='pos1a'
posner_id_b ='pos1b'

IMPORTED MODULES today: 2023-07-18 17:35:57.966886


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_18_01/m_18_01_pos1a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '1a' in file: m_18_01/18_1a_posner_task_2023-06-13_14h28.42.957/18_1a_posner_task_2023-06-13_14h28.42.957.csv
Found '1b' in file: m_18_01/18_1b_posner_task_2023-06-13_15h25.01.529/18_1b_posner_task_2023-06-13_15h25.01.529.csv
********** 1a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-06-13 14:30:43.533229+00:00 - 2023-06-13 14:43:07.253229+00:00
  [0] Keeping  (2023-06-13 14:30:43.533229+00:00 - 2023-06-13 14:30:43.533229+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-06-13 14:31:24.015229+00:00 - 2023-06-13 14:31:24.015229+00:00 -> 40.482 - 4

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [338] Keeping  (2023-06-13 14:40:40.326229+00:00 - 2023-06-13 14:40:40.326229+00:00 -> 596.793 - 596.793)
  [339] Keeping  (2023-06-13 14:40:41.826229+00:00 - 2023-06-13 14:40:41.826229+00:00 -> 598.293 - 598.293)
  [340] Keeping  (2023-06-13 14:40:44.566229+00:00 - 2023-06-13 14:40:44.566229+00:00 -> 601.033 - 601.033)
  [341] Keeping  (2023-06-13 14:40:44.666229+00:00 - 2023-06-13 14:40:44.666229+00:00 -> 601.133 - 601.133)
  [342] Keeping  (2023-06-13 14:40:46.826229+00:00 - 2023-06-13 14:40:46.826229+00:00 -> 603.293 - 603.293)
  [343] Keeping  (2023-06-13 14:40:48.326229+00:00 - 2023-06-13 14:40:48.326229+00:00 -> 604.793 - 604.793)
  [344] Keeping  (2023-06-13 14:40:50.966229+00:00 - 2023-06-13 14:40:50.966229+00:00 -> 607.433 - 607.433)
  [345] Keeping  (2023-06-13 14:40:51.066229+00:00 - 2023-06-13 14:40:51.066229+00:00 -> 607.533 - 607.533)
  [346] Keeping  (2023-06-13 14:40:53.326229+00:00 - 2023-06-13 14:40:53.326229+00:00 -> 609.793 - 609.793)
  [347] Keeping  (2023-06-13

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
413 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 413 events and 701 original time points ...
5 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 41.9s.


Method,fastica
Fit,69 iterations on raw data (663111 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [6]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=663111
    Range : 52660 ... 715770 =     52.660 ...   715.770 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items ac

In [7]:
print(ica.exclude)

[46, 48, 57, 44, 23, 32, 31, 26, 34, 0, 1, 2, 3, 4, 9, 6, 12, 17, 18, 19, 15]


In [8]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 21 ICA components
    Projecting back using 63 PCA components
Using qt as 2D backend.
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [9]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_18_01_pos1a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_18_01

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_18_01/m_18_01_pos1a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [10]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_18_01
├── m_18_01_pos1a.eeg
├── m_18_01_pos1a.vhdr
└── m_18_01_pos1a.vmrk

1 directory, 3 files


In [11]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

In [52]:
### TESTING ###
# test = mne.io.read_raw_brainvision(session_dir+'/'+file_name, preload=True)

In [53]:
# test.plot()

# Posner B

In [12]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_18_01/m_18_01_pos1b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '1a' in file: m_18_01/18_1a_posner_task_2023-06-13_14h28.42.957/18_1a_posner_task_2023-06-13_14h28.42.957.csv
Found '1b' in file: m_18_01/18_1b_posner_task_2023-06-13_15h25.01.529/18_1b_posner_task_2023-06-13_15h25.01.529.csv
********** 1b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Cropping annotations 2023-06-13 15:27:07.558693+00:00 - 2023-06-13 15:39:49.398693+00:00
  [0] Keeping  (2023-06-13 15:27:07.558693+00:00 - 2023-06-13 15:27:07.558693+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-06-13 15:28:01.633693+00:00 - 2023-06-13 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


In [13]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
414 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 414 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [14]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-06-13 15:28:07.447693+00:00 - 2023-06-13 15:39:37.964693+00:00
  [0] Keeping  (2023-06-13 15:28:07.447693+00:00 - 2023-06-13 15:28:07.447693+00:00 -> 59.889 - 59.889)
  [1] Keeping  (2023-06-13 15:28:08.947693+00:00 - 2023-06-13 15:28:08.947693+00:00 -> 61.389 - 61.389)
  [2] Keeping  (2023-06-13 15:28:11.717693+00:00 - 2023-06-13 15:28:11.717693+00:00 -> 64.159 - 64.159)
  [3] Keeping  (2023-06-13 15:28:11.817693+00:00 - 2023-06-13 15:28:11.817693+00:00 -> 64.259 - 64.259)
  [4] Keeping  (2023-06-13 15:28:13.947693+00:00 - 2023-06-13 15:28:13.947693+00:00 -> 66.389 - 66.389)
  [5] Keeping  (2023-06-13 15:28:15.447693+00:00 - 2023-06-13 15:28:15.447693+00:00 -> 67.889 - 67.889)
  [6] Keeping  (2023-06-13 15:28:18.677693+00:00 - 2023-06-13 15:28:18.677693+00:00 -> 71.119 - 71.119)
  [7] Keeping  (2023-06-13 15:28:18.777693+00:00 - 2023-06-13 15:28

Method,fastica
Fit,81 iterations on raw data (690517 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [16]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [16]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=690517
    Range : 59889 ... 750405 =     59.889 ...   750.405 secs
Ready.
Using pyopengl with version 3.1.6


In [17]:
ica_b.exclude

[42, 44, 53, 58, 56, 45, 41, 22, 26, 36, 38, 39, 34, 0, 1, 10, 12, 2, 17, 19]

In [18]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 20 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [19]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_18_01/m_18_01_pos1b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [20]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_18_01
├── m_18_01_pos1a.eeg
├── m_18_01_pos1a.vhdr
├── m_18_01_pos1a.vmrk
├── m_18_01_pos1b.eeg
├── m_18_01_pos1b.vhdr
└── m_18_01_pos1b.vmrk

1 directory, 6 files


# ...

In [21]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b